In [1]:
from pyspark.sql import SparkSession
import pandas

In [24]:
spark = SparkSession.builder.appName("Titanic-Dataframe").getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [25]:
data = spark.read.option("header", True).csv('titanic.csv')

In [26]:
data.take(3)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S')]

In [27]:
df = pandas.read_csv('titanic.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


```SQL
SELECT Name, Sex, Age
FROM Titanic;
```

In [28]:
df[['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
0,"Braund, Mr. Owen Harris",male,22.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
4,"Allen, Mr. William Henry",male,35.0


In [29]:
data.select(data.Name, data.Sex, data.Age).take(5)

[Row(Name='Braund, Mr. Owen Harris', Sex='male', Age='22'),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Allen, Mr. William Henry', Sex='male', Age='35')]

In [30]:
data.select(['Name', 'Sex', 'Age']).take(5)

[Row(Name='Braund, Mr. Owen Harris', Sex='male', Age='22'),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Allen, Mr. William Henry', Sex='male', Age='35')]

```SQL
SELECT Name, Sex, Age * 12 AS AgeMonths
FROM Titanic;
```

In [31]:
df.assign(AgeMonths=df.Age * 12)[['Name', 'Sex', 'AgeMonths']].head(5)

,Name,Sex,AgeMonths
0,"Braund, Mr. Owen Harris",male,264.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,456.0
2,"Heikkinen, Miss. Laina",female,312.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,420.0
4,"Allen, Mr. William Henry",male,420.0


In [32]:
data.withColumn('AgeMonths', data.Age * 12).select('Name', 'Sex', 'AgeMonths').take(5)

[Row(Name='Braund, Mr. Owen Harris', Sex='male', AgeMonths=264.0),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', AgeMonths=456.0),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', AgeMonths=312.0),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', AgeMonths=420.0),
 Row(Name='Allen, Mr. William Henry', Sex='male', AgeMonths=420.0)]

```SQL
SELECT Name, Sex, Age
FROM Titanic
WHERE Survived;
```

In [33]:
df[df.Survived == 1][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0


In [34]:
df.loc[df.Survived == 1, ['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0


In [35]:
data.filter(data.Survived == '1').select('Name', 'Sex', 'Age').take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14')]

```SQL
SELECT Name, Sex, Age
FROM Titanic
WHERE (Survived AND Cabin IS NOT NULL) OR Sex = 'female';
```

In [36]:
df[
    ((df.Survived == 1) & (~df.Cabin.isna())) | (df.Sex == 'female')
][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0


In [37]:
data.filter(
    ((data.Survived == '1') & (~data.Cabin.isNull())) | (data.Sex == 'female')
).select(['Name', 'Sex', 'Age']).take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14')]

```SQL
SELECT Survived, avg(Age)
FROM Titanic
GROUP BY Survived
```

In [38]:
df.groupby('Survived').aggregate({'Age': 'mean'})

,Age
Survived,
0,30.626179
1,28.343690


In [39]:
data.groupBy('Survived').agg({'Age': 'avg'}).collect()

[Row(Survived='0', avg(Age)=30.62617924528302),
 Row(Survived='1', avg(Age)=28.343689655172415)]

```SQL
SELECT Name, Sex, Age
FROM Titanic
WHERE Name LIKE '.*Mrs\.*';
```

In [40]:
df[df.Name.str.match('.*Mrs\.*')][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0
15,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0


In [41]:
data.filter(data.Name.rlike('.*Mrs\.*')).select(['Name', 'Sex', 'Age']).take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14'),
 Row(Name='Hewlett, Mrs. (Mary D Kingcome) ', Sex='female', Age='55')]

In [42]:
data.select(data.Name).take(5)

[Row(Name='Braund, Mr. Owen Harris'),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)'),
 Row(Name='Heikkinen, Miss. Laina'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)'),
 Row(Name='Allen, Mr. William Henry')]

In [43]:
import pyspark.sql.functions as F

data.select(
    F.explode(
        F.split(data.Name, ' ')
    )
).take(20)

[Row(col='Braund,'),
 Row(col='Mr.'),
 Row(col='Owen'),
 Row(col='Harris'),
 Row(col='Cumings,'),
 Row(col='Mrs.'),
 Row(col='John'),
 Row(col='Bradley'),
 Row(col='(Florence'),
 Row(col='Briggs'),
 Row(col='Thayer)'),
 Row(col='Heikkinen,'),
 Row(col='Miss.'),
 Row(col='Laina'),
 Row(col='Futrelle,'),
 Row(col='Mrs.'),
 Row(col='Jacques'),
 Row(col='Heath'),
 Row(col='(Lily'),
 Row(col='May')]

In [44]:
data.rdd.flatMap(lambda row: row.Name.split(' ')).take(20)

['Braund,',
 'Mr.',
 'Owen',
 'Harris',
 'Cumings,',
 'Mrs.',
 'John',
 'Bradley',
 '(Florence',
 'Briggs',
 'Thayer)',
 'Heikkinen,',
 'Miss.',
 'Laina',
 'Futrelle,',
 'Mrs.',
 'Jacques',
 'Heath',
 '(Lily',
 'May']

In [45]:
from pyspark.sql.types import Row

data.rdd.flatMap(lambda row: row.Name.split(' ')) \
    .map(lambda word: Row(name_part = word)) \
    .toDF().take(20)

[Row(name_part='Braund,'),
 Row(name_part='Mr.'),
 Row(name_part='Owen'),
 Row(name_part='Harris'),
 Row(name_part='Cumings,'),
 Row(name_part='Mrs.'),
 Row(name_part='John'),
 Row(name_part='Bradley'),
 Row(name_part='(Florence'),
 Row(name_part='Briggs'),
 Row(name_part='Thayer)'),
 Row(name_part='Heikkinen,'),
 Row(name_part='Miss.'),
 Row(name_part='Laina'),
 Row(name_part='Futrelle,'),
 Row(name_part='Mrs.'),
 Row(name_part='Jacques'),
 Row(name_part='Heath'),
 Row(name_part='(Lily'),
 Row(name_part='May')]